<h2>Team Members</h2>
<ul>
<li>Sohad Hossam Eldin 1190019</li>
<li>Bassant Hisham Mohamed 1190018</li>
<li>Yasmin Hashem Niazy 4200013</li>
<li>Mary Ashraf 1190322</li>
</ul>

<h4>Imports</h4>

In [1]:
from imports import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sohad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<h3>Dataset class<h3>

In [2]:
class ArabicDataset(Dataset):

    def __init__(self, path):

        dataset_txt = open(path, "r", encoding='utf-8').read()
        #print(type(dataset_txt))

        chars_to_remove = r"\(\s*[a-zA-Z0-9_-`]+\s*/\s*[a-zA-Z0-9_-`]+\s*\)|[a-zA-Z0-9_-]+|-|`|–|~|\u200f|'" 
        dataset_cleaned = re.sub(chars_to_remove, "", dataset_txt)
        dataset_sentences = re.split(r"\s*\.\s*|\n|\s*،\s*|\s*:\s*|\s*[()]\s*|\s*؛\s*|\s*؟\s*|\s*!\s*|\s*\"\s*|\s*\[\s*|\s*\]\s*|\s*{\s*|\s*}\s*|\s*\*\s*|\s*»\s*|\s*«\s*|\s*\\\s*|\s*/\s*|\s*;\s*|\s*,\s*",dataset_cleaned)

        self.x_train_letters,self.y_train_letters,self.x_train_words,self.y_train_words, y_vocab = self.Tokenizing(dataset_sentences)

        word_index = {word: idx + 1 for idx, word in enumerate(y_vocab)}
        y_train_indices = [[word_index[word] for word in sentence] for sentence in self.y_train_words]
        self.y_train_padded = tf.keras.preprocessing.sequence.pad_sequences(y_train_indices, padding='post')
        

        self.harakat  = {
                            "": 0,       # No Diacritic
                            "َ": 1,      # Fatha
                            "ً": 2,      # Fathatah
                            "ُ": 3,      # Damma
                            "ٌ": 4,      # Dammatan
                            "ِ": 5,      # Kasra
                            "ٍ": 6,      # Kasratan
                            "ْ": 7,      # Sukun
                            "ّ": 8,      # Shaddah
                            "َّ": 9,     # Shaddah + Fatha
                            "ًّ": 10,    # Shaddah + Fathatah
                            "ُّ": 11,    # Shaddah + Damma
                            "ٌّ": 12,    # Shaddah + Dammatan
                            "ِّ": 13,    # Shaddah + Kasra
                            "ٍّ": 14,    # Shaddah + Kasratan
                            
                        }
        
        self.arabic_alphabet = {
                                "ا": 1,
                                "ب": 2,
                                "ت": 3,
                                "ث": 4,
                                "ج": 5,
                                "ح": 6,
                                "خ": 7,
                                "د": 8,
                                "ذ": 9,
                                "ر": 10,
                                "ز": 11,
                                "س": 12,
                                "ش": 13,
                                "ص": 14,
                                "ض": 15,
                                "ط": 16,
                                "ظ": 17,
                                "ع": 18,
                                "غ": 19,
                                "ف": 20,
                                "ق": 21,
                                "ك": 22,
                                "ل": 23,
                                "م": 24,
                                "ن": 25,
                                "ه": 26,
                                "و": 27,
                                "ي": 28,
                                "آ": 29,
                                "إ": 30,
                                "ئ": 31,
                                "ء": 32,
                                "أ": 33,
                                "ؤ":34,
                                "ة":35,
                                "ى":36,
                            }

        # self.y_train_letters_numbered = LabelEncoder().fit_transform(y_train_letters)
        # self.y_train_words_numbered = LabelEncoder().fit_transform(y_train_words)

        # #dictionary of the labels(words,letters) before converted to numbers
        # self.encoding_mapping_y_letters = {encoded_label: original_label for encoded_label, original_label  in zip(self.y_train_letters_numbered, y_train_letters)}
        # self.encoding_mapping_y_words = {encoded_label: original_label for encoded_label, original_label  in zip( self.y_train_words_numbered, y_train_words)}  

        # print(self.y_train_letters_numbered.shape)
        # print(self.y_train_words_numbered.shape)
        # print( self.y_train_letters_numbered[:10])
        # print( self.y_train_words_numbered[:10])

        # self.y_train_letters_numbered = self.y_train_letters_numbered.reshape((len(self.y_train_letters_numbered), 1))
        # self.y_train_words_numbered = self.y_train_words_numbered.reshape((len(self.y_train_words_numbered), 1))

        # print(self.y_train_letters_numbered.shape)
        # print(self.y_train_words_numbered.shape)
        # print( self.y_train_letters_numbered[:10])
        # print( self.y_train_words_numbered[:10])


    def Tokenizing(self,dataset_sentences):

        x_train_letters = []
        y_train_letters = []
        x_train_words = []
        y_train_words = []
        y_vocab = set()
        for sentence in dataset_sentences:
            sentence_wihout_diacratics = strip_tashkeel(sentence)
            tokens = word_tokenize(sentence, language="arabic", preserve_line=True)
            tokens_wihtout_diacratics = word_tokenize(sentence_wihout_diacratics, language="arabic", preserve_line=True)

            for word in tokens:
                text, inputs, diacritics =util.extract_haraqat(word)
                # inputs.insert(0, "<s>")
                # inputs.append("</s>")

                # diacritics.insert(0, "<s>")
                # diacritics.append("</s>")
                
                x_train_letters.append(inputs)
                y_train_letters.extend(diacritics)


            if(len(tokens)) :
                tokens.insert(0, "<s>")
                tokens.append("</s>")
                y_train_words.append(tokens)
                y_vocab.update(tokens)
            if(len(tokens_wihtout_diacratics)) :
                tokens_wihtout_diacratics.insert(0, "<s>")
                tokens_wihtout_diacratics.append("</s>")
                x_train_words.append(tokens_wihtout_diacratics)

        print(y_train_words[0:10])
        print(y_train_letters[0:10])
        print(x_train_letters[0:10])
        return x_train_letters,y_train_letters,x_train_words,y_train_words, y_vocab

    def __len__(self):
        return len(self.x_train_letters), len(self.x_train_words)

    # get a row at an index
    # The __getitem__ function loads and returns a sample from the dataset at the given index idx
    def __getitem__(self, idx):
        return self.x_train_letters[idx], self.y_train_letters_numbered[idx], self.x_train_words[idx], self.y_train_words_numbered[idx]

    # get indexes for train and test rows
    # def get_splits(self, n_test=0.33):
    #     # determine sizes
    #     test_size = round(n_test * len(self.X))
    #     train_size = len(self.X) - test_size
    #     # calculate the split
    #     return random_split(self, [train_size, test_size])

<h4>Reading the dataset</h4>

In [3]:
# dataset_txt = open(r"train.txt", "r", encoding='utf-8').read()
# print(type(dataset_txt))
#print(dataset_txt)

<h4>Cleaning the dataset</h4>

In [4]:
# do we need to remove [ :ص]
# chars_to_remove = r"\(\s*[a-zA-Z0-9_-]+\s*/\s*[a-zA-Z0-9_-]+\s*\)|[a-zA-Z0-9_-]+" 
# dataset_cleaned = re.sub(chars_to_remove, "", dataset_txt)
#print(dataset_cleaned)

<h4>Segmenting dataset into sentences</h4>

In [5]:
# r'(\([^)]*\))' should we remove the brackets or not
# dataset_sentences = re.split(r"\s*\.\s*|\n|\s*،\s*|\s*:\s*|\s*[()]\s*|\s*؛\s*|\s*؟\s*|\s*!\s*|\s*\"\s*",dataset_cleaned)
# print(type(dataset_sentences))
# print(dataset_sentences[0:30])

<h4>Tokenizing the sentence</h4>

In [6]:
# x_train_letters = []
# y_train_letters = []
# x_train_words = []
# y_train_words = []
# for sentence in dataset_sentences:
#     sentence_wihout_diacratics = strip_tashkeel(sentence)
#     tokens = word_tokenize(sentence, language="arabic", preserve_line=True)
#     tokens_wihtout_diacratics = word_tokenize(sentence_wihout_diacratics, language="arabic", preserve_line=True)
   
#     for word in tokens:
#         text, inputs, diacritics =util.extract_haraqat(word)
#         x_train_letters.append(inputs)
#         y_train_letters.append(diacritics)

#     if(len(tokens)) :
#         tokens.insert(0, "<s>")
#         tokens.append("</s>")
#         y_train_words.append(tokens)
#     if(len(tokens_wihtout_diacratics)) :
#         tokens_wihtout_diacratics.insert(0, "<s>")
#         tokens_wihtout_diacratics.append("</s>")
#         x_train_words.append(tokens_wihtout_diacratics)

## tokenizing the words

In [7]:
# print(x_train_letters[0:10])
# print(y_train_letters[0:10])
# print(y_train_words[0:10])
# print(x_train_words[0:10])

In [8]:
# print(type(x_train_letters))
# print(x_train_letters[2])

#ouh lala , tokinze is dividing the tashkelat as indpendent letters as well , what does that tell us tho 

In [9]:
# temp = ArabicDataset(r"train.txt") 

# Word2vec Feature Extraction

In [10]:
dataset = ArabicDataset(r"train.txt")
# (272126, 255)
# (272126, 255, 20)

[['<s>', 'قَوْلُهُ', '</s>'], ['<s>', 'أَوْ', 'قَطَعَ', 'الْأَوَّلُ', 'يَدَهُ', 'إلَخْ', '</s>'], ['<s>', 'قَالَ', 'الزَّرْكَشِيُّ', '</s>'], ['<s>', 'ابْنُ', 'عَرَفَةَ', '</s>'], ['<s>', 'قَوْلُهُ', '</s>'], ['<s>', 'بِلَفْظٍ', 'يَقْتَضِيه', 'كَإِنْكَارِ', 'غَيْرِ', 'حَدِيثٍ', 'بِالْإِسْلَامِ', 'وُجُوبَ', 'مَا', 'عُلِمَ', 'وُجُوبُهُ', 'مِنْ', 'الدِّينِ', 'ضَرُورَةً', '</s>'], ['<s>', 'كَإِلْقَاءِ', 'مُصْحَفٍ', 'بِقَذَرٍ', 'وَشَدِّ', 'زُنَّارٍ', '</s>'], ['<s>', 'ابْنُ', 'عَرَفَةَ', '</s>'], ['<s>', 'قَوْلُ', 'ابْنِ', 'شَاسٍ', '</s>'], ['<s>', 'أَوْ', 'بِفِعْلٍ', 'يَتَضَمَّنُهُ', 'هُوَ', 'كَلُبْسِ', 'الزُّنَّارِ', 'وَإِلْقَاءِ', 'الْمُصْحَفِ', 'فِي', 'صَرِيحِ', 'النَّجَاسَةِ', 'وَالسُّجُودِ', 'لِلصَّنَمِ', 'وَنَحْوِ', 'ذَلِكَ', '</s>']]
['َ', 'ْ', 'ُ', 'ُ', 'َ', 'ْ', 'َ', 'َ', 'َ', '']
[['ق', 'و', 'ل', 'ه'], ['أ', 'و'], ['ق', 'ط', 'ع'], ['ا', 'ل', 'أ', 'و', 'ل'], ['ي', 'د', 'ه'], ['إ', 'ل', 'خ'], ['ق', 'ا', 'ل'], ['ا', 'ل', 'ز', 'ر', 'ك', 'ش', 'ي'], ['ا', 'ب', 'ن'], ['ع', 'ر', 'ف', 'ة'

In [11]:
# print(dataset.y_train_letters_numbered[:10])
# print(dataset.y_train_words_numbered[:10])
# print(dataset.x_train_words[:10])
# print(dataset.x_train_letters[:10])

model = Word2Vec(sentences=dataset.x_train_words, vector_size=100, window=5, min_count=1, workers=4)

model.save("my_word2vec_model")

# specific_word = 'قوله'
# print(f"{specific_word}: {model.wv[specific_word]}")

# applying Bi-LSTM

In [12]:
word2vec_model = Word2Vec.load("my_word2vec_model")

# + 1 is added so that the padding has a representation in the embedding matrix
embedding_matrix = np.zeros((len(word2vec_model.wv.index_to_key) + 1, word2vec_model.vector_size))

print(word2vec_model.wv.index_to_key[0])

word_index = {word: idx + 1 for idx, word in enumerate(word2vec_model.wv.index_to_key)}
for word, idx in word_index.items():
    embedding_matrix[idx] = word2vec_model.wv[word]

# embedding_matrix = word2vec_model.wv.vectors
# print(embedding_matrix.shape)
print(embedding_matrix.shape)
print(len(word2vec_model.wv.index_to_key) + 1, word2vec_model.vector_size)

x_train_indices = [[word_index[word] for word in sentence] for sentence in dataset.x_train_words]
x_train_padded = tf.keras.preprocessing.sequence.pad_sequences(x_train_indices, padding='post')

# # print(x_train_padded[0])
bi_lstm_model = Sequential()
bi_lstm_model.add(Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1], weights=[embedding_matrix], input_length=x_train_padded.shape[1], trainable=False))
first_bidirectional_layer = Bidirectional(LSTM(units=10, return_sequences=True))
bi_lstm_model.add(first_bidirectional_layer)
# bi_lstm_model.add(Dense(255, activation='sigmoid'))

# bi_lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# bi_lstm_model.fit(x_train_padded, dataset.y_train_padded, epochs=1, batch_size=32)
lstm_output = bi_lstm_model.predict(x_train_padded)
print(type(lstm_output))

# word2vec_model.vector_size --> this is the same as saying embedding_matrix.shape[1]
# keep into considiration the trainable = true parameter
# there is a possibility to pad with the end delimeter

<s>
(105734, 100)
105734 100

8701/8701 [==============================] - 272s 31ms/step
<class 'numpy.ndarray'>


In [13]:
# print(lstm_output[1][0:12])
# print(x_train_padded.shape)

## testing second model for char level encoding

In [14]:
# looping over lstm_output to remove unwanted feature vectors
no_of_sentences, length_of_sentence, feature_vector_size = lstm_output.shape

lstm_output_without_padding = list()
for i in range(no_of_sentences):
    lstm_output_without_padding.extend(lstm_output[i, 1:len(dataset.x_train_words[i])-1, :])

# print(lstm_output_without_padding[0:6])

one_hot_enconding = np.eye(37)
# x_train_letter_concatinated = [[np.concatenate(one_hot_enconding[letter_index], lstm_output_without_padding[i]) for letter_index in x_train_letter_indices[i]] for i in range(len(dataset.x_train_letters))]

x_train_letter_concatenated = []

for i in range(len(dataset.x_train_letters)):
    concatenated_arrays = []
    for letter in dataset.x_train_letters[i]:
        concatenated_arrays.append(np.concatenate([one_hot_enconding[dataset.arabic_alphabet[letter]], lstm_output_without_padding[i]]))
    x_train_letter_concatenated.extend(concatenated_arrays)


#[['ق', 'و', 'ل', 'ه'], ['أ', 'و'], ['ق', 'ط', 'ع'], ['ا', 'ل', 'أ', 'و', 'ل']]

In [15]:
y_train_letter = [dataset.harakat[diacritic] for diacritic in dataset.y_train_letters]

## constructing model 2

In [16]:
bi_lstm_model_char = Sequential()
first_bidirectional_layer = Bidirectional(LSTM(units=64, return_sequences=True))
bi_lstm_model_char.add(Dense(255, activation='softmax'))

bi_lstm_model_char.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

bi_lstm_model_char.fit(np.array(x_train_letter_concatenated), np.array(y_train_letter), epochs=10, batch_size=32)
score = bi_lstm_model_char.evaluate(np.array(x_train_letter_concatenated), np.array(y_train_letter))
print("Test accuracy:", score[1])

bi_lstm_model_char.save("char-model")

In [ ]:
print(x_train_padded.shape)

In [ ]:
word_input = Input(shape=(max_word_length,))
word_embedding = Embedding(input_dim=num_words, output_dim=embedding_dim)(word_input)
word_lstm = Bidirectional(LSTM(units=lstm_units, return_sequences=True))(word_embedding)